In [2]:
# loading pretrained inception V3 model for generating deep dream images

import keras
from keras.applications import inception_v3
from keras import backend as K

# We will not be training our model,
# so we use this command to disable all training-specific operations
K.set_learning_phase(0)

# Build the InceptionV3 network.
# The model will be loaded with pre-trained ImageNet weights.
model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

87916544/87910968 [==============================] - 14s    


In [3]:
# setting the deepdream config, i.e. what layers should contribute to the final image
# this is a somewhat arbitraty selection but can be changed and experimented with

# Dict mapping layer names to a coefficient quantifying how much the layer's activation
# will contribute to the loss we will seek to maximize.
# Note that these are layer names as they appear in the built-in InceptionV3 application.
# You can list all layer names using `model.summary()`.
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

In [4]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_95 (Conv2D)               (None, None, None, 32 864         input_2[0][0]                    
____________________________________________________________________________________________________
batch_normalization_95 (BatchNor (None, None, None, 32 96          conv2d_95[0][0]                  
____________________________________________________________________________________________________
activation_95 (Activation)       (None, None, None, 32 0           batch_normalization_95[0][0]     
___________________________________________________________________________________________

In [5]:
# defininig a loss by the weighted sum of L2 norm of the activations of the layers defined above

# Get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Define the loss.
loss = K.variable(0.) #instantiating a variable with value 0
for layer_name in layer_contributions:
    # Add the L2 norm of the features of a layer to the loss.
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    # We avoid border artifacts by only involving non-border pixels in the loss.
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
layer_dict

{'input_2': <keras.engine.topology.InputLayer at 0x7fac0037e5f8>,
 'conv2d_95': <keras.layers.convolutional.Conv2D at 0x7fac0037e940>,
 'batch_normalization_95': <keras.layers.normalization.BatchNormalization at 0x7fac0037e7b8>,
 'activation_95': <keras.layers.core.Activation at 0x7fac0038d2e8>,
 'conv2d_96': <keras.layers.convolutional.Conv2D at 0x7fac0038d9b0>,
 'batch_normalization_96': <keras.layers.normalization.BatchNormalization at 0x7fac0037ecf8>,
 'activation_96': <keras.layers.core.Activation at 0x7fac003a5780>,
 'conv2d_97': <keras.layers.convolutional.Conv2D at 0x7fac003a5b70>,
 'batch_normalization_97': <keras.layers.normalization.BatchNormalization at 0x7fac002fe898>,
 'activation_97': <keras.layers.core.Activation at 0x7fac00313b38>,
 'max_pooling2d_5': <keras.layers.pooling.MaxPooling2D at 0x7fac00313c18>,
 'conv2d_98': <keras.layers.convolutional.Conv2D at 0x7fac002cbc88>,
 'batch_normalization_98': <keras.layers.normalization.BatchNormalization at 0x7fac002832b0>,
 'a

In [7]:
# Same as in making visible the layers activation in chapter 5, we define a gradient ascent process.

# This holds our generated image
dream = model.input

# Compute the gradients of the dream with regard to the loss.
grads = K.gradients(loss, dream)[0]

# Normalize gradients --> important trick
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# Set up function to retrieve the value
# of the loss and gradients given an input image.
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [8]:
# defining auxiliary numpy functions for the deepdream algorithm

import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)


def preprocess_image(image_path):
    # Util function to open, resize and format pictures
    # into appropriate tensors.
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [9]:
# We start by defining a list of scales at which to process the image.
# Then we process a small image with the deepdream algorithm, scale it up by 40% and go on like this.

import numpy as np

# Playing with these hyperparameters will also allow you to achieve new effects

step = 0.01  # Gradient ascent step size
num_octave = 3  # Number of scales at which to run gradient ascent
octave_scale = 1.4  # Size ratio between scales
iterations = 20  # Number of ascent steps per scale

# If our loss gets larger than 10,
# we will interrupt the gradient ascent process, to avoid ugly artifacts
max_loss = 10.

# Fill this to the path to the image you want to use
base_image_path = '/home/joel/DL_Python/generative_dl/IMG_3405.JPG'

# Load the image into a Numpy array
img = preprocess_image(base_image_path)

# We prepare a list of shape tuples
# defining the different scales at which we will run gradient ascent
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# Reverse list of shapes, so that they are in increasing order
successive_shapes = successive_shapes[::-1]

# Resize the Numpy array of the image to our smallest scale
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')

/home/joel/anaconda3/lib/python3.7/site-packages/scipy/ndimage/interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Processing image shape (1175, 1763)
...Loss value at 0 : 1.2755861
...Loss value at 1 : 1.9073415
...Loss value at 2 : 2.7064319
...Loss value at 3 : 3.645239
...Loss value at 4 : 4.581405
...Loss value at 5 : 5.4915633
...Loss value at 6 : 6.370616
...Loss value at 7 : 7.231332
...Loss value at 8 : 8.059938
...Loss value at 9 : 8.857846
...Loss value at 10 : 9.642599


/home/joel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Processing image shape (1645, 2468)
...Loss value at 0 : 3.703653
...Loss value at 1 : 5.2247214
...Loss value at 2 : 6.412139
...Loss value at 3 : 7.46295
...Loss value at 4 : 8.454355
...Loss value at 5 : 9.386311
Processing image shape (2304, 3456)
...Loss value at 0 : 3.5648026
...Loss value at 1 : 4.989954
...Loss value at 2 : 6.160798
...Loss value at 3 : 7.2511234
...Loss value at 4 : 8.305636
...Loss value at 5 : 9.311065
